In [3]:
#!pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pyarrow

In [12]:
# path = "/content/drive/My Drive/dataEngProject/kym_vision.json"

SyntaxError: invalid syntax (Temp/ipykernel_7476/3082006881.py, line 2)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_json("kym_vision.json", orient="index")

In [3]:
df["url"] = df.index
df = df.reset_index()
df = df.drop(columns=["index"])
df.head()

,labelAnnotations,safeSearchAnnotation,webDetection,error,url
0,"[{'mid': '/m/01yrx', 'description': 'Cat', 'sc...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/04wpw', 'sco...",NaN,https://knowyourmeme.com/memes/this-is-relevan...
1,"[{'mid': '/m/0dzct', 'description': 'Face', 's...","{'adult': 'UNLIKELY', 'spoof': 'LIKELY', 'medi...","{'webEntities': [{'entityId': '/m/03bzkh4', 's...",NaN,https://knowyourmeme.com/memes/leave-britney-a...
2,"[{'mid': '/m/068jd', 'description': 'Photograp...","{'adult': 'UNLIKELY', 'spoof': 'VERY_LIKELY', ...","{'webEntities': [{'entityId': '/m/09dzg', 'sco...",NaN,https://knowyourmeme.com/memes/i-like-turtles
3,"[{'mid': '/m/09j2d', 'description': 'Clothing'...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/09kqc', 'sco...",NaN,https://knowyourmeme.com/memes/bitches-dont-know
4,"[{'mid': '/m/04hgtk', 'description': 'Head', '...","{'adult': 'VERY_UNLIKELY', 'spoof': 'VERY_LIKE...","{'webEntities': [{'entityId': '/m/06ng23', 'sc...",NaN,https://knowyourmeme.com/memes/o-rly


In [4]:
safeSearchToNumberCode = {
    "VERY_UNLIKELY": 0.0,
    "UNLIKELY": 0.25,
    "POSSIBLE": 0.5,
    "LIKELY": 0.75,
    "VERY_LIKELY": 1.0
}
def getSafeSearch(row):
  if (isinstance(row.safeSearchAnnotation, dict)):
    for key in row.safeSearchAnnotation.keys():
      row["safeSearch_"+key] = safeSearchToNumberCode[row.safeSearchAnnotation[key]]
  return row

In [5]:
df = df.apply(getSafeSearch, axis=1)
df = df.drop(columns=["safeSearchAnnotation"])

In [6]:


with open("kym_vision_processed.json", "w", encoding="utf-8") as f:
    f.write(df.to_json(orient='records', lines=True))
    


In [5]:
df2 = pd.read_json("kym_vision_processed.json", orient="records", lines=True)


KeyError: 1

In [8]:
df2["webDetection"][0]

{'webEntities': [{'entityId': '/m/04wpw',
   'score': 0.7058,
   'description': 'Meme'},
  {'entityId': '/m/02pqt5g', 'score': 0.7044, 'description': 'Lolcat'},
  {'entityId': '/m/09g6pyy',
   'score': 0.6384000000000001,
   'description': 'Know Your Meme'},
  {'entityId': '/m/025rxrz', 'score': 0.6216, 'description': 'Internet meme'},
  {'entityId': '/m/09kqc', 'score': 0.6038, 'description': 'Humour'},
  {'entityId': '/m/0ryr6fh', 'score': 0.5448000000000001},
  {'entityId': '/m/02jjt', 'score': 0.5149, 'description': 'Entertainment'},
  {'entityId': '/m/0jg24', 'score': 0.5106, 'description': 'Image'},
  {'entityId': '/m/09v48rh', 'score': 0.4454, 'description': 'Imgur'},
  {'entityId': '/m/043zsw', 'score': 0.3849, 'description': 'Curiosity'},
  {'entityId': '/m/01qbld', 'score': 0.3806, 'description': 'LOL'},
  {'entityId': '/m/015hcf', 'score': 0.3779, 'description': 'Idea'},
  {'entityId': '/m/033lpr',
   'score': 0.36910000000000004,
   'description': 'Audience'},
  {'entityId'

In [8]:
df2.to_parquet("kym_vision_processed.parquet")

In [10]:
df3 = pd.read_parquet('kym_vision_processed.parquet', engine='pyarrow')
df3.head()

,error,labelAnnotations,safeSearch_adult,safeSearch_medical,safeSearch_racy,safeSearch_spoof,safeSearch_violence,url,webDetection
0,None,"[{'description': 'Cat', 'mid': '/m/01yrx', 'sc...",0.00,0.00,0.00,1.00,0.25,https://knowyourmeme.com/memes/this-is-relevan...,{'bestGuessLabels': [{'label': 'interest meme'...
1,None,"[{'description': 'Face', 'mid': '/m/0dzct', 's...",0.25,0.25,0.25,0.75,0.25,https://knowyourmeme.com/memes/leave-britney-a...,{'bestGuessLabels': [{'label': 'leave britney ...
2,None,"[{'description': 'Photograph', 'mid': '/m/068j...",0.25,0.25,0.00,1.00,0.25,https://knowyourmeme.com/memes/i-like-turtles,"{'bestGuessLabels': [{'label': 'like turtles',..."
3,None,"[{'description': 'Clothing', 'mid': '/m/09j2d'...",0.00,0.00,0.25,1.00,0.25,https://knowyourmeme.com/memes/bitches-dont-know,{'bestGuessLabels': [{'label': 'eric harris fu...
4,None,"[{'description': 'Head', 'mid': '/m/04hgtk', '...",0.00,0.25,0.00,1.00,0.50,https://knowyourmeme.com/memes/o-rly,"{'bestGuessLabels': [{'label': 'o rly', 'langu..."


In [8]:
def getLabelAnnotScores(row):
  if ((not type(row.labelAnnotations) == float) and len(row.labelAnnotations) > 0):
    newDict = {annot["description"]: annot["score"] for annot in row.labelAnnotations}
    for key in newDict.keys():
        row["annot_"+key] = newDict[key]
  return row

In [ ]:
def getBestGuessLabel(row):
  if ((type(row.webDetection) == dict)):
    # and "bestGuessLabels" in row.webDetection and len(row.webDetection.bestGuessLabels) > 0):
    bestGuessLabels = row.webDetection.get("bestGuessLabels", [])
    if (len(bestGuessLabels) > 0 and 'label' in bestGuessLabels[0]):
      row["webDetection_bestGuessLabel"] = bestGuessLabels[0]['label']
  return row

In [ ]:
df = df.apply(getLabelAnnotScores, axis=1)
df = df.drop(columns="labelAnnotations")

In [ ]:
df = df.apply(getBestGuessLabel, axis=1)
df = df.drop(columns="webDetection")

In [ ]:
df.head()

,annot_3d modeling,annot_3x3 (basketball),annot_800 metres,annot_A-line,annot_Abdomen,annot_Academic certificate,annot_Academic dress,annot_Academic institution,annot_Accessory fruit,annot_Accipitridae,annot_Accipitriformes,annot_Accordion,annot_Accordionist,annot_Acoustic guitar,annot_Acoustic-electric guitar,annot_Acrobatics,annot_Acrylic paint,annot_Acting,annot_Action figure,annot_Action film,annot_Action-adventure game,annot_Active pants,annot_Active shirt,annot_Active shorts,annot_Active tank,annot_Adaptation,annot_Address sign,annot_Adhesive,annot_Admiral,annot_Adventure,annot_Advertising,annot_Aeolian landform,annot_Aerial photography,annot_Aerialist,annot_Aerobatics,annot_Aerospace engineering,annot_Aerospace manufacturer,annot_African elephant,annot_African leopard,annot_Afro,...,annot_heater,annot_horse flies,annot_house fly,annot_jean short,annot_kangaroo,annot_lorikeet,annot_meadow jumping mouse,annot_millipedes,annot_octopus,annot_packrats,annot_pear,annot_pipistrelles,annot_robber flies,annot_rubber ducky,annot_rugby short,annot_shortstraw pine,annot_signaling device,annot_stable fly,annot_studio couch,annot_sunflower,annot_sweatpant,annot_trailer truck,annot_trick-or-treat,annot_wasp,annot_water lily,annot_white footed mice,annot_wild cabbage,annot_wild carrot,annot_wood rabbit,annot_yoga pant,annot_À la carte food,annot_Épée,error,safeSearch_adult,safeSearch_medical,safeSearch_racy,safeSearch_spoof,safeSearch_violence,url,webDetection_bestGuessLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,1.00,0.25,https://knowyourmeme.com/memes/this-is-relevan...,interest meme
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.25,0.25,0.75,0.25,https://knowyourmeme.com/memes/leave-britney-a...,leave britney alone
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.508471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,0.25,0.00,1.00,0.25,https://knowyourmeme.com/memes/i-like-turtles,like turtles
3,NaN,NaN,NaN,NaN,0.556868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.663647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.25,1.00,0.25,https://knowyourmeme.com/memes/bitches-dont-know,eric harris funny
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.25,0.00,1.00,0.50,https://knowyourmeme.com/memes/o-rly,o rly


In [ ]:
df.to_parquet("/content/drive/My Drive/dataEngProject/kym_vision_processed.parquet")

In [ ]:
len(df.columns)

4730

In [ ]:
df.shape

(16686, 4730)